<a href="https://colab.research.google.com/github/somilasthana/MachineLearningSkills/blob/master/NLP_TopicModeling_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# get data
!wget https://liferay.de.dariah.eu/tatom/_downloads/datasets.zip

--2019-06-24 09:17:54--  https://liferay.de.dariah.eu/tatom/_downloads/datasets.zip
Resolving liferay.de.dariah.eu (liferay.de.dariah.eu)... 134.76.30.131
Connecting to liferay.de.dariah.eu (liferay.de.dariah.eu)|134.76.30.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57659024 (55M) [application/zip]
Saving to: ‘datasets.zip’

datasets.zip        100%[===================>]  54.99M  17.9MB/s    in 3.8s    

2019-06-24 09:17:58 (14.5 MB/s) - ‘datasets.zip’ saved [57659024/57659024]



In [0]:
!unzip /content/datasets.zip > /dev/null

In [0]:
import os

CORPUS_PATH = "/content/data/austen-brontë-split/"
import pandas as pd

In [0]:
filenames = sorted([os.path.join(CORPUS_PATH, fn) for fn in os.listdir(CORPUS_PATH)])

In [37]:
filenames[:5]

['/content/data/austen-brontë-split/Austen_Emma0000.txt',
 '/content/data/austen-brontë-split/Austen_Emma0001.txt',
 '/content/data/austen-brontë-split/Austen_Emma0002.txt',
 '/content/data/austen-brontë-split/Austen_Emma0003.txt',
 '/content/data/austen-brontë-split/Austen_Emma0004.txt']

In [0]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
vectorizer = CountVectorizer(input='filename', stop_words='english', min_df=20)

In [0]:
dtm = vectorizer.fit_transform(filenames).toarray()

In [0]:
vocab = np.array(vectorizer.get_feature_names())

In [13]:
dtm.shape

(813, 2902)

In [0]:
from sklearn import decomposition

num_topics = 20
num_topic_words = 20

In [0]:
clf = decomposition.NMF(n_components=num_topics, random_state=42)

In [0]:
doctopic = clf.fit_transform(dtm)

In [0]:
topic_words = []

for topic in clf.components_:
  word_idx = np.argsort(topic)[::-1][0:num_topic_words]
  topic_words.append(vocab[word_idx])

In [35]:
for t in range(len(topic_words)):
  print("Topic {}: {}".format(t, ' '.join(topic_words[t][:15])))

Topic 0: know think like man say good said does oh quite sure yes thing believe way
Topic 1: mr knightley rochester elton man did think evening business away wickham gentleman time wife told
Topic 2: marianne elinor willoughby sister jennings mother colonel dashwood mrs brandon time moment heart little barton
Topic 3: elizabeth darcy bingley mr sister bennet wickham miss soon lydia friend room gardiner little netherfield
Topic 4: mrs did quite elton room come good thing said weston jennings came day party palmer
Topic 5: like room door night long saw thought looked house time eyes day face rochester light
Topic 6: sir rochester yes shall adele don little good john ll said mr say eyre room
Topic 7: jane fairfax love little think oh heard heart mother soon make feel voice campbell bates
Topic 8: miss woodhouse bates quite sure young fairfax oh lady say room smith temple bessie great
Topic 9: john man good father house mother years young dashwood life old family wife home woman
Topic 10: 

In [0]:
"""
we will scale the document-component matrix such that the component values 
associated with each document sum to one.
"""

doctopic = doctopic / np.sum(doctopic, axis=1, keepdims=True)

In [0]:
novel_names = []
for fn in filenames:
  basename = os.path.basename(fn)
  name, ext = os.path.splitext(basename)
  name = name.rstrip('0123456789')
  novel_names.append(name)

In [0]:
novel_names = np.array(novel_names)

In [39]:
novel_names[:5]

array(['Austen_Emma', 'Austen_Emma', 'Austen_Emma', 'Austen_Emma',
       'Austen_Emma'], dtype='<U17')

In [0]:
doctopic_orig = doctopic.copy()

num_groups = len(set(novel_names))

doctopic_grouped = np.zeros((num_groups, num_topics))

for i, name in enumerate(sorted(set(novel_names))):
  doctopic_grouped[i, :] = np.mean(doctopic[novel_names == name, : ], axis=0)

In [33]:
pd.DataFrame(doctopic_grouped, index=sorted(set(novel_names)), columns=["Topic"+str(i) for i in range(20)])

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19
Austen_Emma,0.057720,0.119328,0.002480,0.004690,0.097054,0.017695,0.010075,0.056122,0.088726,0.051078,0.009967,0.007426,0.009594,0.127265,0.047962,0.001776,0.070894,0.005166,0.153811,0.061171
Austen_Pride,0.042627,0.092781,0.006355,0.252878,0.053054,0.020050,0.013831,0.057689,0.038718,0.039022,0.005000,0.127313,0.009209,0.002781,0.058804,0.001078,0.073294,0.007558,0.001717,0.096242
Austen_Sense,0.041137,0.030601,0.237698,0.005904,0.097010,0.020937,0.027894,0.003421,0.039017,0.083471,0.007152,0.016278,0.006909,0.003119,0.053376,0.001254,0.084082,0.179137,0.002112,0.059492
CBronte_Jane,0.041017,0.081824,0.001511,0.001541,0.043616,0.214324,0.095344,0.072396,0.057230,0.068022,0.023135,0.009204,0.036051,0.001674,0.073756,0.005739,0.113147,0.004186,0.002376,0.053906
CBronte_Professor,0.059571,0.043067,0.003022,0.002990,0.012514,0.187182,0.008961,0.008669,0.002341,0.052686,0.019048,0.010750,0.302158,0.004292,0.084533,0.036350,0.089236,0.012331,0.002417,0.057883
CBronte_Villette,0.055043,0.005893,0.001900,0.002015,0.025776,0.140051,0.005526,0.003987,0.025660,0.051799,0.136727,0.009927,0.094034,0.004818,0.114780,0.144811,0.106430,0.015157,0.003117,0.052550


In [0]:
# Find Distinct Topics

novels = sorted(set(novel_names))
austen_indices=[0, 1, 2]
austen_avg = np.mean(doctopic[austen_indices, :], axis=0)
cbronte_indices = [3, 4, 5]
cbronte_avg = np.mean(doctopic[cbronte_indices, :], axis=0)

keyness = np.abs(austen_avg - cbronte_avg)
ranking = np.argsort(keyness)[::-1]

In [41]:
ranking[:10]

array([ 4,  9,  0, 16,  8, 18,  1,  5, 13, 12])

In [0]:
doctopic_grouped = np.zeros((num_groups, 10))

for i, name in enumerate(sorted(set(novel_names))):
  for j,rank in enumerate(ranking[:10]):
    doctopic_grouped[i, j] = np.mean(doctopic[novel_names == name, rank ], axis=0)

In [64]:
pd.DataFrame(doctopic_grouped, index=sorted(set(novel_names)), columns=["Topic"+str(i) for i in ranking[:10]])

,Topic4,Topic9,Topic0,Topic16,Topic8,Topic18,Topic1,Topic5,Topic13,Topic12
Austen_Emma,0.097054,0.051078,0.057720,0.070894,0.088726,0.153811,0.119328,0.017695,0.127265,0.009594
Austen_Pride,0.053054,0.039022,0.042627,0.073294,0.038718,0.001717,0.092781,0.020050,0.002781,0.009209
Austen_Sense,0.097010,0.083471,0.041137,0.084082,0.039017,0.002112,0.030601,0.020937,0.003119,0.006909
CBronte_Jane,0.043616,0.068022,0.041017,0.113147,0.057230,0.002376,0.081824,0.214324,0.001674,0.036051
CBronte_Professor,0.012514,0.052686,0.059571,0.089236,0.002341,0.002417,0.043067,0.187182,0.004292,0.302158
CBronte_Villette,0.025776,0.051799,0.055043,0.106430,0.025660,0.003117,0.005893,0.140051,0.004818,0.094034


In [0]:
np_topic_words = np.array(topic_words)



In [66]:
pd.DataFrame(np_topic_words[ranking[:10]], index=["Topic"+str(i) for i in ranking[:10]])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Topic4,mrs,did,quite,elton,room,come,good,thing,said,weston,jennings,came,day,party,palmer,fairfax,colonel,went,know,heard
Topic9,john,man,good,father,house,mother,years,young,dashwood,life,old,family,wife,home,woman,fortune,sisters,thousand,make,st
Topic0,know,think,like,man,say,good,said,does,oh,quite,sure,yes,thing,believe,way,love,young,people,better,thought
Topic16,said,come,shall,know,tell,think,john,cried,looked,did,replied,let,asked,answer,door,yes,answered,quite,want,eyes
Topic8,miss,woodhouse,bates,quite,sure,young,fairfax,oh,lady,say,room,smith,temple,bessie,great,came,just,think,heard,ladies
Topic18,emma,weston,mr,knightley,frank,churchill,thing,little,father,woodhouse,great,fairfax,time,hartfield,body,randalls,soon,mrs,dear,bates
Topic1,mr,knightley,rochester,elton,man,did,think,evening,business,away,wickham,gentleman,time,wife,told,dear,gentlemen,brother,information,quite
Topic5,like,room,door,night,long,saw,thought,looked,house,time,eyes,day,face,rochester,light,heard,little,stood,hand,hour
Topic13,harriet,emma,elton,mr,friend,woodhouse,martin,knightley,think,good,thing,little,smith,thought,mind,woman,hartfield,just,great,having
Topic12,monsieur,little,english,mdlle,frances,mademoiselle,vous,est,said,pelet,hand,pupils,eyes,reuter,henri,thought,time,school,french,good
